In [1]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
from helpers.rotation import RotateFromTo

from yaml import load
from yaml.loader import Loader


In [2]:
BASENAME = "16_Archamps"

fp = f"../../real_data/{BASENAME}/params.yaml"

params = load(open(fp), Loader)

GLOBAL_SHIFT = params["alignment"]["globalShift"]

PASSAGE = "Meandre"
SECTION = 1

In [3]:
data = pv.read(f"../../real_data/{BASENAME}/output/{PASSAGE}_{SECTION}_clean.stl").extract_surface().triangulate()

type(data)

pyvista.core.pointset.PolyData

In [16]:
plane_origins = np.loadtxt(f"../../real_data/{BASENAME}/process/planes_as_vectors/{PASSAGE}_{SECTION}_Planes_Origin.txt") + GLOBAL_SHIFT
plane_vectors =  np.loadtxt(f"../../real_data/{BASENAME}/process/planes_as_vectors/{PASSAGE}_{SECTION}_Planes.txt")

In [17]:
# define orthogonal unit vectors in horizontal plane 
d0 = np.array([1,0,0])
d1 = np.array([0,1,0])

# project the section-normal vectors onto horizontal plane. 
horiz_dip_dir = np.asarray((plane_vectors.dot(d0), plane_vectors.dot(d1), np.zeros(len(plane_vectors)))).T

In [18]:
print(len(horiz_dip_dir))

39


In [19]:
plane_origins += np.array([100, 0,100])

In [20]:
plane_origins

array([[ 62.63697445, -48.82138707,  41.07440339],
       [ 62.2774407 , -49.16825281,  41.08639185],
       [ 61.93217751, -49.52895092,  41.10198298],
       [ 61.63457907, -49.93032971,  41.0906935 ],
       [ 61.35561459, -50.34448129,  41.06567662],
       [ 61.06536248, -50.74870889,  41.01863405],
       [ 60.75302758, -51.12760488,  40.92584453],
       [ 60.43312464, -51.49329526,  40.80786657],
       [ 60.1107195 , -51.86250079,  40.71023852],
       [ 59.79727548, -52.24990381,  40.68214766],
       [ 59.50319622, -52.65188945,  40.72301253],
       [ 59.19854618, -53.04240983,  40.79022333],
       [ 58.8471943 , -53.39279351,  40.84656726],
       [ 58.45734747, -53.70052925,  40.90314629],
       [ 58.06113164, -53.99989904,  40.96110746],
       [ 57.70238121, -54.34193448,  41.02235144],
       [ 57.2735897 , -54.58363548,  41.03989091],
       [ 56.78556054, -54.67814739,  41.06856625],
       [ 56.30420607, -54.72014577,  41.19594912],
       [ 55.82542758, -54.74826

In [21]:
data.cast_to_pointset().points 

pyvista_ndarray([[ 54.815674, -55.98761 ,  39.248257],
                 [ 54.815674, -56.020573,  39.242764],
                 [ 54.8042  , -56.020573,  39.248257],
                 ...,
                 [ 61.253826, -49.11972 ,  41.753765],
                 [ 61.22691 , -49.096226,  42.09402 ],
                 [ 61.248337, -49.11493 ,  41.860588]], dtype=float32)

In [22]:
4 * data.volume / data.area

2.0367658501048136

In [23]:
for n, (orig, vec, vec_horiz) in tqdm(enumerate(zip(plane_origins, plane_vectors, horiz_dip_dir)), total = len(plane_origins)):
    plane = pv.Plane(orig, vec, 20, 20, 100, 100).extract_surface().triangulate()
    inter, _, _ = data.intersection(plane, split_first = False, split_second = False)

    R1 =  RotateFromTo(vec, vec_horiz, verbose=False)
    pp = inter.points - np.mean(plane.points, axis=0)
    pp_vertical = R1 @ pp.T
    R2 = RotateFromTo(vec_horiz, d1)
    pp_rotated = R2 @ pp_vertical

    np.savetxt(f"../../real_data/{BASENAME}/process/sections_as_polylines/{PASSAGE}_{SECTION}_unordered_section_{n:03d}.txt", pp_rotated.T)
        
    #pp_reordered = order_points(pp_rotated.T.tolist(), 0)


100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:15<00:00,  2.49it/s]
